# Description

In this exercise, we continue to work with the small "numbertheory" library used in prior exercises. However, a new function has been added for this exercise. This lendd itself to testing with mocks. Let us look at it:

```python
def get_primes_between(start=0, end=100):
    "Find all primes in interval [start, end]"
    assert n < m
    url = f'http://example.org/primes-between/json/{start}/{end}'
    response = requests.get(url)
    return response.json()
```

The function is not really needed for primes in a small range where our existing functions could easily generate these answers.  However, for larger numbers, the load on the local system might be appropriate to transfer to a dedicated service.  E.g., we might do this:

```python
>>> get_primes_between(961_748_940, 961_748_988)
[961748941, 961748947, 961748951, 961748969, 961748987]
```

This question can be answered *relatively* feasibly using `likely_prime()`, but for a larger range it becomes more difficult.  Performing 1.5 billion Miller-Rabin tests (assuming 40 trials per candidate) can be *very slow*.

```python
>>> get_primes_between(961_748_988, 999_999_999)
[961748993, 961749023, 961749037, ...approx 2 million more...]
```

Adding to the issue is that the server we intend to use does not actually even exist (yet).  Using mocks or stubs, do the following:

* Use local calculation to simulate getting a moderate number of moderately large primes.  E.g. a few hundred in the 1,000,000-1,500,000 range at a time.
* Use a custom mocked object that will provide an approximation of the *number* of items expected from a call.

# Setup

In [1]:
import pytest
import unittest
from numbertheory.utilities import *

def test_exact_moderate_range():
    # Check for correct results for moderate result size
    # Ideal test should be parameterized to check a number of ranges
    correct = [1_000_003, 1_000_033, 1_000_037,
               1_000_039, 1_000_081, 1_000_099]
    assert get_primes_between(1_000_000, 1_000_100) == correct
    

def test_approx_large_range():
    # Check for approximate results for large result size
    # Ideal test should be parameterized to check a number of ranges
    primes = get_primes_between(100_000_000, 105_000_000)
    at_least, at_most = 279_682, 338_416
    assert at_least < len(primes) < at_most

# Solution

In [2]:
import pytest
import requests
#from utilities import *

class JSONAnswer:
    def __init__(self, data):
        self.data = data
        
    def json(self):
        return self.data

def mock_get(url):
    parts = url.split('/')
    start, stop = map(int, parts[-2:])
    primes = get_primes_upto(stop)
    result = [n for n in primes if n >= start]
    return JSONAnswer(result)
        
                          
def test_exact_moderate_range(monkeypatch):
    # Check for correct results for moderate result size
    correct = [1_000_003, 1_000_033, 1_000_037,
               1_000_039, 1_000_081, 1_000_099]
    monkeypatch.setattr(requests, 'get', mock_get)
    assert get_primes_between(1_000_000, 1_000_100) == correct


class JSONLen:
    def __init__(self, len_):
        self.len = len_
        
    def json(self):
        class Len:
            def __init__(self, len_):
                self.len = len_
            def __len__(self):
                return self.len
            
        return Len(self.len)

def mock_get_approx(url):
    parts = url.split('/')
    start, stop = map(int, parts[-2:])
    return JSONLen(prime_count(stop) - prime_count(start))
    
    
def test_approx_large_range(monkeypatch):
    # Check for approximate results for large result size
    monkeypatch.setattr(requests, 'get', mock_get_approx)
    primes = get_primes_between(100_000_000, 105_000_000)
    at_least, at_most = 279_682, 338_416
    assert at_least < len(primes) < at_most

# Test Cases

In [3]:
def test_enough_tests():
    tests = {name for name in globals() if name.startswith('test_')}
    for name in ['test_exact_moderate_range', 
                 'test_approx_large_range']:
        assert name in tests
    
test_enough_tests()